# El metodo de encriptacion RSA
Tenemos el siguiente escenario: Roberto quiere mandar un mensaje a Alicia de tal forma que culquier persona distinta a Roberto que se encuentre en la transmision no pueda entender el mensaje. Esto se logra encriptando el mensaje para que solo Alicia, con la clave correcta, pueda descifrarlo.

La idea es que Alicia escoja dos primos distintos de longitud grande $p,q$, y fija $n=pq$. Cualquier persona que pueda factorizar $n$ podra acceder al mensaje.

Si Alicia quiere recibir mensajes de Roberto, ella escoje $e\in\{2,\dots,\varphi(n)-2\}$ con $\text{mcd}(e,\varphi(n))=1$ aleatoriamente, donde $\varphi$ es la funcion $\varphi$ de Euler, ademas al ser $p,q$ primos $\varphi(n)=(p-1)(q-1)$. (Tambien puede fijar $e=3$). Despues calcula $d\in\{2,\dots,\varphi(n)-2\}$ con $de\equiv1(\bmod{\varphi(n)})$, usando el algoritmo de Euclides y expandiendolo como combinacion lineal, publica la pareja $K=(n, e)$ esta sera su clave publica y mantiene tanto su clave privada $S=(n, d)$ como $p,q$ ocultas.

Para mandar un mensaje numerico $m$ a Alicia, Roberto utiliza la clave publica de Alicia y lo encripta mediante $\hat{m}=m^e$, y lo envia a Alicia, que calcula $m^d$, usando su clave privada. Entonces, usando un $u\in\mathbb{Z}$ tal que $de-1=u\cdot\varphi(n)$ (Inverso), obtenemos $$m^{de}=m^{1+u\cdot\varphi(n)}=m(m^{\varphi(n)})^{u}≡m(\bmod{n}),$$ Puesto que $m^{\varphi(n)}=1$ por el Teorema de Euler .

## Implementacion en Python 1 (Sin librerias y entrada numerica)


Primero creamos una funcion que nos ayude a calcular el MCD de dos numeros usando el algoritmo de Euclides

In [1]:
from random import randint
def MCD_Euclides(a,b):
  while a != b:
    if a > b:
      a = a - b
    else:
      b = b - a
  return a

print(MCD_Euclides(12,8))

4


Ahora una funcion que nos ayude a calcular el inverso modulo $n$

In [2]:
def inverso(e,n):
  # El inverso es un entero tal que e * nbr mod n es 1
  for i in range(n):
    if ((i+1) * e) % n == 1:
      return i+1
  return -1 #No hay inverso

print(inverso(3,11))

4


Con estas funciones previas podemos definir otras que nos sirvan para encriptar y desencriptar nuestro mensaje; y la funcion.

In [3]:
def generar_claves(p, q):
    pq = p * q
    phi = (p - 1) * (q - 1)
    e = 0
    while True:
        e = randint(3, phi)
        if MCD_Euclides(e, phi) == 1:
            break
    d = inverso(e, phi)
    return {'e': e, 'pq': pq}, {'d': d, 'pq': pq}

def encriptar(mensaje, clave_publica):
    e = clave_publica['e']
    pq = clave_publica['pq']
    encriptado = (mensaje ** e) % pq
    return encriptado

def desencriptar(encriptado, clave_privada):
    d = clave_privada['d']
    pq = clave_privada['pq']
    decodificado = (encriptado ** d) % pq
    return decodificado

def main():
    p = 89 #Puede elegir el primo
    q = 13 #Puede elegir el primo
    mensaje = int(input('Ingrese el mensaje numerico a encriptar: '))

    clave_publica, clave_privada = generar_claves(p, q)

    print(f"p:\t{p}\nq:\t{q}\np*q:\t{clave_publica['pq']}\nphi:\t{(p - 1) * (q - 1)}\nMensaje a Encriptar:\t{mensaje}")
    print(f"Se encontro e = {clave_publica['e']}")
    print(f"Se encontro d = {clave_privada['d']}")
    print(f"e: {clave_publica['e']:,} tiene el inverso d: {clave_privada['d']:,} ... ({clave_publica['e']:,} * {clave_privada['d']:,}) % {(p - 1) * (q - 1):,} = {(clave_publica['e'] * clave_privada['d']) % ((p - 1) * (q - 1))}")
    print(f"Entonces la clave publica es: \t{clave_publica}")
    print(f"La clave privada es: \t\t{clave_privada}")

    encriptado = encriptar(mensaje, clave_publica)
    decodificado = desencriptar(encriptado, clave_privada)

    print(f'Encriptando: {mensaje:,}\nEncriptado: {encriptado:,}\nDecodificado: {decodificado:,}')

if __name__ == "__main__":
    main()

Ingrese el mensaje numerico a encriptar: 245
p:	89
q:	13
p*q:	1157
phi:	1056
Mensaje a Encriptar:	245
Se encontro e = 955
Se encontro d = 115
e: 955 tiene el inverso d: 115 ... (955 * 115) % 1,056 = 1
Entonces la clave publica es: 	{'e': 955, 'pq': 1157}
La clave privada es: 		{'d': 115, 'pq': 1157}
Encriptando: 245
Encriptado: 639
Decodificado: 245


**NOTA:** A pesar de que esta implementacion sea facilmente legible, puede llegar a fallar por errores de sobreflujo, es decir si el mensaje a encriptar es un entero muy grande decodifica otro numero

# Implementacion 2 en Python (Librerias y entrada en cadena)
Comenzaremos con dos funciones, la primera checara si algun numero es primo y la otra generara primos

In [4]:
import random

def esPrimo(numero):
  if numero <2: #Verifica si el nuemero es menor que 2 para descartarlo
    return False
  for i in range(2, numero // 2 + 1): #Itera a traves de todos los nuemeros impares y verifica si son divisibles entre i para descartarlos
    if numero % i == 0:
      return False
  return True

def generarPrimo(minimo, maximo):
  primo = random.randint(minimo, maximo) #Usando randint genera un numero aleatorio para posteriormente checar si es primo con la funcion previa
  while not esPrimo(primo):
    primo = random.randint(minimo, maximo)
  return primo

Lo siguiente sera realizar una funcion que calcula el inverso modular, y escribir el texto a cifrar.

In [5]:
import math

def inverso_mod(e, phi):
  for d in range(3,phi): # Itera los numeros d tales que 3<d<=phi
    if (d*e) % phi == 1: #Checa la condicion d*e≡1(mod{phi})
      return d
  raise ValueError("El inverso modular no existe")

p, q = generarPrimo(100, 1000), generarPrimo(100, 1000)

while p == q:
  q = generarPrimo(100, 1000)

n = p * q
phi_n = (p-1)*(q-1)

e = random.randint(3, phi_n-1)
while math.gcd(e, phi_n) != 1:
  e = random.randint(3, phi_n-1)

d = inverso_mod(e, phi_n)

print("Clave publica: ", (n, e))
print("Clave privada: ", (n, d))
print("Phi(n):  ", phi_n)
print("p: ", p)
print("q: ", q)

mensaje = input("Ingrese el mensaje a encriptar: ")
mensaje_codificado = [ord(char) for char in mensaje] #Ingresa a una lista el caracter en formato ASCII de la cadena solicitada al usuario

encriptado = [pow(char, e, n) for char in mensaje_codificado] #(m^e) mod n = c
print("Mensaje encriptado: ", encriptado)

desencriptado = [chr(pow(char, d, n)) for char in encriptado] # (c^d) mod n = m
print("Mensaje desencriptado: ", ''.join(desencriptado))


Clave publica:  (368351, 257075)
Clave privada:  (368351, 135419)
Phi(n):   367104
p:  479
q:  769
Ingrese el mensaje a encriptar: Teoria de Conjuntos
Mensaje encriptado:  [305187, 196543, 329791, 173589, 191644, 32291, 202427, 74464, 196543, 202427, 19186, 329791, 28361, 272067, 260398, 28361, 20679, 329791, 134506]
Mensaje desencriptado:  Teoria de Conjuntos


Elaboro:

*   Ruiz Barrera Isaac
*   Cordova Castro Alberto
*   Lopez Brenis Gibran
*   Martinez Castañeda Abner

